<p>Class:  USC Viterbi Data Analytics Bootcamp</p>
<p>Team:  Analyticus (aka Team 5)</p>
<p>Module:  pull_cdc_state_5_years.py</p>
<p>Input:  CDC Influenza-Like-Illness CSV File for last five flu seasons.</p>
<p>Output:  CDC json file with flu cases for the last five flu seasons.</p>

In [1]:
# Import dependances.
import json
import csv
import pandas as pd

In [3]:
# Load CDC data into a dataframe.
df = pd.read_csv('../data/cdc_state_5_years.csv', skiprows=[0])

In [4]:
# Inspect the dataframe.
df.head()

,REGION TYPE,REGION,YEAR,WEEK,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 25-49,AGE 25-64,AGE 5-24,AGE 50-64,AGE 65,ILITOTAL,NUM. OF PROVIDERS,TOTAL PATIENTS
0,States,Alabama,2012,40,X,1.49404,X,X,X,X,X,X,238,39,15930
1,States,Alaska,2012,40,X,1.19893,X,X,X,X,X,X,27,8,2252
2,States,Arizona,2012,40,X,0.756794,X,X,X,X,X,X,132,32,17442
3,States,Arkansas,2012,40,X,0.471698,X,X,X,X,X,X,14,17,2968
4,States,California,2012,40,X,2.10133,X,X,X,X,X,X,720,123,34264


In [5]:
# Select only columns needed.
df2 = df[['YEAR', 'WEEK', 'REGION', 'ILITOTAL']]

In [6]:
# Inspect dataframe
df2.head()

,YEAR,WEEK,REGION,ILITOTAL
0,2012,40,Alabama,238
1,2012,40,Alaska,27
2,2012,40,Arizona,132
3,2012,40,Arkansas,14
4,2012,40,California,720


In [7]:
# Normalize CDC year and weeks to HHS year and weeks.
df3 = pd.DataFrame()
for df2_index in df2.index:
    if df2.loc[df2_index, 'WEEK'] > 39:
        df3.loc[df2_index, 'YEAR'] = df2.loc[df2_index, 'YEAR']
        df3.loc[df2_index, 'WEEK'] = df2.loc[df2_index, 'WEEK'] - 39
    else:
        df3.loc[df2_index, 'WEEK'] = df2.loc[df2_index, 'WEEK'] + 13
        df3.loc[df2_index, 'YEAR'] = df2.loc[df2_index, 'YEAR'] - 1
    df3.loc[df2_index, 'REGION'] = df2.loc[df2_index, 'REGION']
    if df2.loc[df2_index, 'ILITOTAL'] == 'X':
        df3.loc[df2_index, 'ILITOTAL'] = 0
    else:
        df3.loc[df2_index, 'ILITOTAL'] = df2.loc[df2_index, 'ILITOTAL']
    

In [ ]:
df3.head()

In [ ]:
df4 = df3.sort_values(['YEAR', 'WEEK', 'REGION'])

In [ ]:
df4.head()

In [ ]:
df5 = df4

In [ ]:
df5['YEAR'] = df5['YEAR'].astype(int)
df5['WEEK'] = df5['WEEK'].astype(int)
df5['ILITOTAL'] = df5['ILITOTAL'].astype(int)

In [ ]:
df5.head()

In [ ]:
df6 = pd.DataFrame(df5.groupby(['YEAR','REGION']).agg({'ILITOTAL': 'sum'}))

In [ ]:
df6.head()

In [ ]:
for df5_index in df5.index:
    
    year_index = df5.loc[df5_index, 'YEAR']
    region_index = df5.loc[df5_index, 'REGION']
    
    state_total = df6.loc[year_index].loc[region_index][0]
    
    if state_total > 0:
        df5.loc[df5_index, 'FLU_PERCENT'] = (df5.loc[df5_index, 'ILITOTAL'] / state_total) * 100
    else:
        df5.loc[df5_index, 'FLU_PERCENT'] = 0.0


In [ ]:
df7 = df5.loc[(df5['REGION'] == 'Alabama') & (df5['YEAR'] == 2013)]

In [ ]:
df7.head()

In [ ]:
df8 = pd.DataFrame(df7.groupby(['YEAR']).agg({'FLU_PERCENT': 'sum'}))

In [ ]:
df8

In [ ]:
df5.head()

In [ ]:
df9 = df5.rename(columns = {"YEAR":"year", "WEEK":"week", "REGION":"state", "ILITOTAL":"flu_cases", "FLU_PERCENT":"flu_percent"})

In [ ]:
df9.head()

In [ ]:
# Write string to cdc.json
df9.to_json('data/cdc_state_5_years.json')

In [ ]:
dfA = pd.read_json('data/cdc_state_5_years.json')

In [ ]:
dfB = dfA.sort_values(by=['year', 'state', 'week'])

In [ ]:
dfB.head()